In [1]:
import os, fnmatch
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd
import sys
import json
import glob

# Labels
Sources:
- clinical.tsv
- riboDepleted_samples_that_passedQC_and_have_known_diagnosis
- TranscriptMethod_THPEDv1

# Features
Sources:
- /data/archive/compendium/v5/v5_hugo_log2tpm.11340x58581.2018-02-03.hd5
- /data/archive/downstream/*  [all sampleids possible]
    - /secondary/ucsc_cgl-rnaseq-cgl-pipeline-0.0.0-0000000/RSEM/Hugo/
        - rsem_genes.hugo.results

# Gather Features
1. read in compendium (log2(TPM+1))
2. find more in downstream source (TPM) 
3. merge the two using the same gene name both in log2(TPM+1)

In [2]:
%%time
compendium = pd.read_hdf("/data/archive/compendium/v5/v5_hugo_log2tpm.11340x58581.2018-02-03.hd5")

basepath="/data/archive/downstream/"
postidpath="/secondary/ucsc_cgl-rnaseq-cgl-pipeline-0.0.0-0000000/RSEM/Hugo/"
filename="rsem_genes.hugo.results"
hugoLogTpmPlusOneFilePathList = glob.glob(basepath + "*" + postidpath + filename)

rawTPMExpression = pd.DataFrame()
for filepath in hugoLogTpmPlusOneFilePathList:
    curDf = pd.read_csv(filepath,sep='\t')
    rawTPMExpression[filepath[25:].partition('/')[0]]=curDf['TPM']

CPU times: user 3min 9s, sys: 1min 3s, total: 4min 12s
Wall time: 4min 12s


### Multiple Ensemble IDs match to one HUGO gene name

In [3]:
rawTPMExpression['gene_id'] = curDf['gene_name']
rawTPMExpression.index = list(rawTPMExpression['gene_id'])

### Remove repeat genes and apply +1 and log2 to rawTPM

In [4]:
rawTPMExpressionShorter=rawTPMExpression.groupby(level=0).agg('mean').add(1).apply(np.log2)

### Merge compendium with downstream data

In [5]:
%%time
cols_to_use = rawTPMExpressionShorter.columns.difference(compendium.columns)

expressionTpmCompendium = pd.merge(rawTPMExpressionShorter[list(cols_to_use)], compendium, left_index = True, right_index = True)

CPU times: user 1min 25s, sys: 13.5 s, total: 1min 38s
Wall time: 1min 39s


# Gather Labels
1. read clinical.tsv (no riboD, so all polyA)
2. read riboDepleted_samples_that_passedQC_and_have_known_diagnosis (all riboD)
3. read TranscriptMethod_THPEDv1 (compendium both polyA and riboD)
4. merge all three and remove repeats

In [6]:
clinicalLabels = pd.read_csv("/data/archive/compendium/v5/clinical.tsv",sep='\t')
clinicalLabels['tr_method']='PolyA'
ribodDiagnosis = pd.read_csv("riboDepleted_samples_that_passedQC_and_have_known_diagnosis.tsv", sep='\t')

ribodDiagnosis=ribodDiagnosis.rename(columns={'Treehouse SAMPLE identifier':'th_sampleid','Diagnosis/Disease':'disease'})

ribodDiagnosis['tr_method']='RiboMinus'
methods = pd.read_csv("TranscriptMethod_THPEDv1.csv")

methods = methods.rename(columns={'Treehouse SAMPLE identifier':'th_sampleid','TR_method':'tr_method'})

clinicalIdTissue = clinicalLabels[['th_sampleid','anat_sample','disease','tr_method']] 
df = pd.merge(clinicalIdTissue, ribodDiagnosis, how='outer')
df = pd.merge(df,methods,how='outer')
df = df[['th_sampleid','tr_method','disease']]
# PolyA                11350
# RiboMinus              179
# suspect RiboMinus        9

compendium_id_method_disease_labels = df.dropna()
# PolyA        11340
# RiboMinus      165

# 11454 features intersect labels total
- **160 features intersect labels that are RiboMinus**
- 11340 features intersect labels that are PolyA

In [7]:
len(set(list(compendium_id_method_disease_labels['th_sampleid']))&set(expressionTpmCompendium.keys()))

11454

In [8]:
labelsRiboD=compendium_id_method_disease_labels[compendium_id_method_disease_labels['tr_method']=='RiboMinus']['th_sampleid']
len(set(labelsRiboD)&set(expressionTpmCompendium.keys()))

160

In [9]:
labelsRiboD=compendium_id_method_disease_labels[compendium_id_method_disease_labels['tr_method']=='PolyA']['th_sampleid']
len(set(labelsRiboD)&set(expressionTpmCompendium.keys()))

11340

# Make labels and features intersect ids
1. find all columns that are intersecting in both
2. create features from compatible columns
3. remove duplicates from labels 
4. transpose label list to access ids as columns
5. create labels from compatible columns

In [10]:
allColumns = set(compendium_id_method_disease_labels['th_sampleid'])|set(expressionTpmCompendium.keys())
columnsNotInLabels = allColumns^set(compendium_id_method_disease_labels['th_sampleid'])
columnsNotInFeatures = allColumns^set(expressionTpmCompendium.keys())
allNonCompatibleColumns = columnsNotInFeatures^columnsNotInLabels
allCompatibleColumns = allColumns-allNonCompatibleColumns

In [11]:
features = expressionTpmCompendium[list(allCompatibleColumns)]
#  have 11454 columns

In [12]:
compendium_id_method_disease_labels=compendium_id_method_disease_labels.drop_duplicates('th_sampleid')
compendium_id_method_disease_labels.index = compendium_id_method_disease_labels['th_sampleid']
transposeCompendium = compendium_id_method_disease_labels.T
labels = transposeCompendium[list(allCompatibleColumns)]
#  also have 11454 columns

### Test whether the sets have truely the same ids 
(If all true then = proved)

In [13]:
print(set(features.keys())<=set(labels.keys()))
print(set(features.keys())>=set(labels.keys()))
# just to double check
print(set(labels.keys())<=set(features.keys()))
print(set(labels.keys())>=set(features.keys()))

True
True
True
True


# Feature and Label tables 
- 114 RiboMinus samples
- 11340 PolyA samples

In [14]:
labelsTall = labels.T[['tr_method','disease']]

In [15]:
labelsTall.head()

,tr_method,disease
th_sampleid,,
TCGA-BP-4176-01,PolyA,kidney clear cell carcinoma
TCGA-GC-A3BM-01,PolyA,bladder urothelial carcinoma
TCGA-CQ-7063-01,PolyA,head & neck squamous cell carcinoma
TCGA-3K-AAZ8-01,PolyA,hepatocellular carcinoma
TCGA-24-1431-01,PolyA,ovarian serous cystadenocarcinoma


In [16]:
features.head()

,TCGA-BP-4176-01,TCGA-GC-A3BM-01,TCGA-CQ-7063-01,TCGA-3K-AAZ8-01,TCGA-24-1431-01,TCGA-ET-A3DO-01,TARGET-10-PARSGC-03,TCGA-FG-8181-01,TCGA-F5-6811-01,THR29_0755_S01,...,TCGA-28-2509-01,TCGA-Q3-AA2A-01,TCGA-24-2023-01,TCGA-YZ-A984-01,TCGA-41-2572-01,TCGA-DE-A4MD-01,TCGA-DD-AAEH-01,TCGA-B9-A69E-01,TCGA-EY-A5W2-01,TCGA-KL-8328-01
5S_rRNA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5_8S_rRNA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7SK,0.054294,0.000000,0.000000,0.000000,0.033324,0.000000,0.037990,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
A1BG,2.744170,1.384132,2.111051,11.403532,1.895314,4.880677,4.392991,3.659896,1.361835,4.107688,...,5.691269,3.39778,4.395753,2.700467,6.587368,1.339210,10.864874,3.177962,2.980032,1.963548
A1BG-AS1,1.316216,0.536153,0.918464,1.189114,0.815671,2.843969,1.214170,3.542255,0.739941,2.381283,...,2.648508,0.96354,1.546046,1.918422,3.244872,0.443721,0.926076,1.384132,0.669106,0.201757


### Export to csv and hdf5 files

(This cell takes awhile to run)

In [19]:
%%time
labelsTall.to_csv('compendiumLabels.tsv', sep='\t')

features.to_csv('data/compendiumExpression.tsv', sep='\t')

CPU times: user 22min 39s, sys: 1min 56s, total: 24min 35s
Wall time: 24min 38s


# Find Disease with at least one RiboMinus and one PolyA
- For use in DESeq

In [ ]:
allDiseases=list(labelsTall['disease'].unique())

maxRiboMinusDisease = ''
maxRiboD=0
multiMethodDiseases = []
for disease in allDiseases:
    polyA_in_this_disease = list(labelsTall[labelsTall.disease == disease]['tr_method']).count('PolyA')
    riboD_in_this_disease = list(labelsTall[labelsTall.disease == disease]['tr_method']).count('RiboMinus')
    if riboD_in_this_disease >= 1 and polyA_in_this_disease >= 1:
        multiMethodDiseases.append(disease)
print(multiMethodDiseases)


### Above is for DESeq analysis<br>
Below, I am going to be finding disease specific data and also use more variable genes to create my feature set for the first Logistic Regression. 

# Find Disease with the most RiboMinus

In [ ]:
allDiseases=list(labelsTall['disease'].unique())

maxRiboMinusDisease = ''
maxRiboD=0
for disease in allDiseases:
    polyA_in_this_disease = list(labelsTall[labelsTall.disease == disease]['tr_method']).count('PolyA')
    riboD_in_this_disease = list(labelsTall[labelsTall.disease == disease]['tr_method']).count('RiboMinus')
    if riboD_in_this_disease > maxRiboD:
        maxRiboMinusDisease = disease
        maxRiboD = riboD_in_this_disease
print(maxRiboMinusDisease)

# Find Disease with closest 1/1 RiboMinus to PolyA
- looks for highest riboD count and
- highest riboD/polyA ratio

In [ ]:
mostProportionalDisease = ''
bestRatio=0.0
highestRiboD = 0
for disease in allDiseases:
    polyA_in_this_disease = list(labelsTall[labelsTall.disease == disease]['tr_method']).count('PolyA')
    riboD_in_this_disease = list(labelsTall[labelsTall.disease == disease]['tr_method']).count('RiboMinus')
    if polyA_in_this_disease==0: polyA_in_this_disease=1
    if riboD_in_this_disease==0: riboD_in_this_disease=1
    ratio=riboD_in_this_disease/polyA_in_this_disease
    if ratio < 1 :
        if bestRatio < ratio and riboD_in_this_disease > highestRiboD:
            bestRatio = ratio
            highestRiboD = riboD_in_this_disease
            mostProportionalDisease = disease
print(mostProportionalDisease, bestRatio, highestRiboD)
print('polya:',list(labelsTall[labelsTall.disease == mostProportionalDisease]['tr_method']).count('PolyA'))
print('ribominus:',list(labelsTall[labelsTall.disease == mostProportionalDisease]['tr_method']).count('RiboMinus'))


# Create most highly variating genes (not DESeq)
- Get 75th highest variable genes
- Make features have the same genes as those above 75th percentile

This takes a long time, you can just read in the variation calculation from a csv file

In [ ]:
%%time
variationFeatures = features.var(axis=1,numeric_only=True)

Read in from csv file

In [ ]:
variationFeatures = pd.read_csv("variationFeatures.csv",header=0,index_col=0)

variationFeatures = variationFeatures.to_frame(name='var')

variationFeatures.to_csv(path_or_buf="variationFeatures.csv", sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=None, date_format=None, doublequote=True, escapechar=None, decimal='.')

Get 75th highest variable genes

In [ ]:
highVarGenes = variationFeatures[variationFeatures > variationFeatures.quantile(q=0.75)].dropna()

len(list(highVarGenes.index))

Make features have only these genes

In [ ]:
%%time
highVarFeatures = features[features.index.isin(list(highVarGenes.index))]

# Glioblastoma Multiforme Features and Labels
- one feature csv with high variance genes (14,000 genes)
- one feature csv with all genes (58,000 genes)
- one label csv with identical identifiers (polyA riboD method)
- 233 samples
- 192 polya
- 41 ribod

In [ ]:
th_ids_glioma = list(labelsTall[labelsTall.disease == "glioblastoma multiforme"].index)

gliomaFeaturesVar = highVarFeatures[th_ids_glioma]
gliomaFeatures = features[th_ids_glioma]
gliomaLabels = labelsTall[labelsTall.disease == "glioblastoma multiforme"]
# Export csv format files
gliomaFeaturesVar.to_csv("data/glioblastomaExpression14kgenes.csv")
gliomaFeatures.to_csv("data/glioblastomaExpression.csv")
gliomaLabels.to_csv("data/glioblastomaLabels.csv")

# Export h5 format files
with pd.HDFStore("data/glioblastomaTrain.h5", "w") as store:
    store["expression"] = gliomaFeatures.T.sort_index(axis="columns")
    store["labels"] = gliomaLabels.astype(str)

# Create Small Test Set of Another Disease
- Look for the next best 1/1 riboD polyA with more than 10 riboD samples
- get the features and labels to test on model

In [ ]:
mostProportionalDisease = ''
bestRatio=0.0
highestRiboD = 10
for disease in allDiseases:
    polyA_in_this_disease = list(labelsTall[labelsTall.disease == disease]['tr_method']).count('PolyA')
    riboD_in_this_disease = list(labelsTall[labelsTall.disease == disease]['tr_method']).count('RiboMinus')
    if polyA_in_this_disease==0: polyA_in_this_disease=1
    if riboD_in_this_disease==0: riboD_in_this_disease=1
    ratio=riboD_in_this_disease/polyA_in_this_disease
    if ratio < 1 :
        if bestRatio < ratio and riboD_in_this_disease > highestRiboD:
            bestRatio = ratio
            highestRiboD = riboD_in_this_disease
            mostProportionalDisease = disease
            print(mostProportionalDisease, bestRatio, highestRiboD)
            print('polya:',list(labelsTall[labelsTall.disease == mostProportionalDisease]['tr_method']).count('PolyA'))
            print('ribominus:',list(labelsTall[labelsTall.disease == mostProportionalDisease]['tr_method']).count('RiboMinus'))


acute lymphoblastic leukemia is the next most highly riboD and lowest ratio disease

# Create Labels for Test Set

Create Features and Labels for Test Set and export to csv files and hdf5

In [ ]:
%%time
th_ids_ALL = list(labelsTall[labelsTall.disease=='acute lymphoblastic leukemia'].index)
ALL_FeaturesVar = highVarFeatures[th_ids_ALL]
ALL_Features = features[th_ids_ALL]
ALL_Labels = labels[list(ALL_Features.T.index)]

# Export csv format files
ALL_FeaturesVar.to_csv("data/ALLeukemiaExpressionVar.csv")
ALL_Features.to_csv("data/ALLeukemiaExpression.csv")
ALL_Labels.to_csv("data/ALLeukemiaLabels.csv")

In [ ]:
# Export h5 format files
with pd.HDFStore("data/ALLeukemiaTrain.h5", "w") as store:
    store["expression"] = ALL_Features.T.sort_index(axis="columns")
    store["labels"] = ALL_Labels.astype(str)

# Reference

In [ ]:
df2 = pd.DataFrame(np.random.randint(low=0, high=10, size=(5, 5)),columns=['a', 'b', 'c', 'd', 'e'])
df1 = pd.DataFrame(np.random.randint(low=10, high=20, size=(5, 5)),columns=['a', 'b', 'c', 'd', 'e'])

pd.concat([df2,df1]).sort_index()

pd.concat([df2,df1]).sort_index().groupby(level=0).agg('mean')